In [61]:
"""Script to solve scalar wave equation on Schwarzschild spacetime. Practice for the gravitational scattering case. The detailed \
physical setup is described by Barack & Burko (2000) (https://arxiv.org/abs/gr-qc/0007033)]. Essentially, we consider a scalar charge \
q plunging into a Schwarzschild Black Hole of mass M and calculate the first order self-force correction to the small object's motion \
as a perturbation to motion on the Schwarzschild background."""

"Script to solve scalar wave equation on Schwarzschild spacetime. Practice for the gravitational scattering case. The detailed physical setup is described by Barack & Burko (2000) (https://arxiv.org/abs/gr-qc/0007033)]. Essentially, we consider a scalar charge q plunging into a Schwarzschild Black Hole of mass M and calculate the first order self-force correction to the small object's motion as a perturbation to motion on the Schwarzschild background."

In [62]:
import time
from datetime import timedelta
import numpy as np
import scipy.special as scsp
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

In [63]:
%matplotlib widget

In [64]:
# For the first part of the problem, we simply analyse how a perturbation to the initial field configuration evolves by scattering off the 
# Schwarzschild background in a vacuum region of the spacetime. In particular, we find that the initial perturbation, and accompanying junk 
# radiation both decay exponentially in vacuum over time.

In [65]:
class scalar_field:
    """A class for the initialisation and evolution of a scalar field on a Schwarzschild background, given some initial radial coordinate r_in
    and some maximal radial coordinate r_max along with the step size."""

    def tort_coord(self, r):
        """Return of the tortoise coordinate for a given input radial coordinate and Schwarzschild BH mass"""
        M = self.BH_mass
        return r + 2*M*np.log(r/(2*M) - 1)
    
    def __init__(self, r_in, r_max, h):
        """Instantiate the scalar field with the initial and maximal radial coordinates, the mass of the background Schwarzschild BH, initial and 
        maximal tortoise coordinates, maximal time coordinate, and the number of steps used to construct the grid."""
        M = 1.0
        self.BH_mass = M
        self.r_init = r_in
        self.r_max = r_max
        self.step_size = h

        # Calculate the initial and maximal values of the tortoise coordinates.
        self.tort_init = self.tort_coord(r_in)
        self.tort_max = self.tort_coord(r_max)

        # Calculate the maximal value of the time coordinate. For a square (v, u) grid, this amounts to t_max - t_min = tort_max - tort_min, where
        # we choose t_min = 0 by convention. Use this to calculate the number of steps required to go from 0 to t_max (this is because numpy's 
        # arange is unstable when the endpoints with large separation, so we use linspace instead, which requires num_step instead of step_size. 
        # On the other hand, the initial data specifies step_size since this is a more physically relevant parameter to control, for e.g. when 
        # carrying out convergence tests).
        self.t_max = self.tort_max - self.tort_init
        self.num_step = 1 + int(self.t_max/self.step_size)

    def time_array(self):
        """Return the array of time instants, taking t_in = 0."""
        t_max = self.t_max
        num_step = self.num_step

        return np.linspace(0, t_max, num_step)

    def v_u_coord(self):
        """Return the arrays describing the v and u axes."""
        num_step = self.num_step
        v_in = self.tort_init
        u_in = -self.tort_init
        v_max = self.t_max + self.tort_init
        u_max = self.t_max - self.tort_init

        v_arr = np.linspace(v_in, v_max, num_step)
        u_arr = np.linspace(u_in, u_max, num_step)

        return v_arr, u_arr

    def rad_val_arr(self, v_arr, u_arr):
        """Computes the values of the radial coordinate r (not tortoise!) corresponding to each (v, u) point, which can then be fed into 
        the effective potential. Since the grid is square in v & u, and since the null coordinates are monotonic functions of t and r, the 
        diagonals of the (v, u) square grid correspond to lines of constant t and constant r. For instance, the line connecting 
        (v_in, u_in) and (v_max, u_max) is a line of constant r, with only t increasing. Thus from the v and u axes, we can construct an r 
        axis, which is a 1D array (instead of a 2D array computing r for each (v, u)) of 2*(len(u))-1 elements encoding the various radial
        values needed for the computation."""
        M = self.BH_mass
        num_step = self.num_step
        tort_init = self.tort_init
        tort_max = self.tort_max
        r_arr = np.zeros(2*len(u_arr) - 1)

        # The first half of r_arr can be computed by fixing v = v[0] and varying u over the range of its values. The second half can be computed
        # by fixing u = u[0] and varying v, this time from v[1] (to avoid double counting the r element corresponding to (u[0], v[0])).
        for i in range(len(u_arr)):
            v0 = v_arr[0]
            u0 = u_arr[0]
            r_arr[len(u_arr) - 1 - i] = 2*M*(np.real(scsp.lambertw(np.exp(-1 + (v0 - u_arr[i])/(4*M)))) + 1)
            r_arr[len(u_arr) - 1 + i] = 2*M*(np.real(scsp.lambertw(np.exp(-1 + (v_arr[i] - u0)/(4*M)))) + 1)

        return r_arr

    def field_evo(self):
        v_arr, u_arr = self.v_u_coord()
        field = np.zeros((len(v_arr), len(u_arr)))

        return field

In [66]:
r_in = 20.0
r_max = 1000.0
h = 2
psi = scalar_field(r_in, r_max, h)

In [67]:
psi.BH_mass, psi.r_init, psi.r_max, psi.step_size

(1.0, 20.0, 1000.0, 2)

In [68]:
psi.tort_init, psi.tort_max, psi.t_max, psi.num_step

(np.float64(24.39444915467244),
 np.float64(1012.4252121915031),
 np.float64(988.0307630368306),
 495)

In [69]:
v_range, u_range = psi.v_u_coord()

In [70]:
len(v_range), len(u_range)

(495, 495)

In [71]:
rad_arr = psi.rad_val_arr(v_range, u_range)
len(rad_arr)

989

In [72]:
rad_arr[:]

array([  2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.        ,
         2.        ,   2.        ,   2.        ,   2.  

In [73]:
v_range[0], u_range[0]

(np.float64(24.39444915467244), np.float64(-24.39444915467244))

In [74]:
rad_arr[0]

np.float64(2.0)

In [75]:
umax = u_range[-1]
umax

np.float64(963.6363138821582)

In [76]:
rmin = 2*(np.real(scsp.lambertw(np.exp(-1 + (v_range[0] - umax)/4))) + 1)
rmin

np.float64(2.0)

In [79]:
vmax = np.max(v_range)
vmax

np.float64(1012.4252121915031)

In [80]:
umax = np.max(u_range)
umax

np.float64(963.6363138821582)

In [81]:
ruvmax = 2*(np.real(scsp.lambertw(np.exp(-1 + (vmax - umax)/4))) + 1)
ruvmax

np.float64(19.999999999999993)

In [82]:
rout = 2*(np.real(scsp.lambertw(np.exp(-1 + (vmax - u_range[0])/4))) + 1)
rout

np.float64(507.34563977350473)

In [83]:
np.max(rad_arr)

np.float64(507.34563977350473)

In [85]:
u0 = u_range[0]
u0

np.float64(-24.39444915467244)